In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [2]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)
train_test = pd.concat([df_train, df_test], axis=0)
df_hist = utils.read_df_pkl('../input/hist_clean*')
df_new = utils.read_df_pkl('../input/new_clean*')

df_train = utils.reduce_mem_usage(df_train)
df_test  = utils.reduce_mem_usage(df_test )
df_hist  = utils.reduce_mem_usage(df_hist )
df_new   = utils.reduce_mem_usage(df_new  )

100%|██████████| 3/3 [00:01<00:00,  2.66it/s]


Mem. usage decreased to  4.04 Mb (56.2% reduction)
Mem. usage decreased to  2.24 Mb (52.5% reduction)
Mem. usage decreased to 2554.26 Mb (0.0% reduction)
Mem. usage decreased to 183.47 Mb (0.0% reduction)


In [3]:
df_hist.head()

,card_id,purchase_date,authorized_flag,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,category_2,state_id,subsector_id,first_active_month,all_term,yyyymm,yyyymmdd,weekofyear,yyyy_week,purchase_amount_over
0,C_ID_4e6213e9bc,2017-06-25 15:33:07,1,88,0,0.0,A,80,M_ID_e020e9b302,-8,0.043579,1.0,16,37,2016-06-01,699,2017-06,2017-06-25,25,2017-25,0.0
1,C_ID_4e6213e9bc,2017-07-15 12:10:45,1,88,0,0.0,A,367,M_ID_86ec983688,-7,0.013786,1.0,16,16,2016-06-01,699,2017-07,2017-07-15,28,2017-28,0.0
2,C_ID_4e6213e9bc,2017-08-09 22:04:29,1,88,0,0.0,A,80,M_ID_979ed661fc,-6,0.026535,1.0,16,37,2016-06-01,699,2017-08,2017-08-09,32,2017-32,0.0
3,C_ID_4e6213e9bc,2017-09-02 10:06:26,1,88,0,0.0,A,560,M_ID_e6d5ae8ea6,-5,0.011566,1.0,16,34,2016-06-01,699,2017-09,2017-09-02,35,2017-35,0.0
4,C_ID_4e6213e9bc,2017-03-10 01:14:19,1,88,0,0.0,A,80,M_ID_e020e9b302,-11,0.024048,1.0,16,37,2016-06-01,699,2017-03,2017-03-10,10,2017-10,0.0


In [3]:
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]
# for df in [df_hist]:
#     df['purchase_date'] = pd.to_datetime(df['purchase_date'])
#     df['weekofyear'] = df['purchase_date'].dt.weekofyear
#     df['month'] = df['purchase_date'].dt.month
#     df['hour'] = df['purchase_date'].dt.hour
#     df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
#     df['date_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)

In [6]:
# df_hist.drop(['city_id', 'category_1', 'installments', 'category_2', 'category_3', 'merchant_category_id', 'merchant_id', 'purchase_amount'], axis=1, inplace=True)
auth1 = df_hist[df_hist['authorized_flag']==1]
auth0 = df_hist[df_hist['authorized_flag']==0]

# df_hist['yyyymm'] =  df_hist['purchase_date'].map(lambda x: str(x)[:7] + '-01')
# df_hist['yyyy_week'] = df_hist['yyyymm'].map(lambda x: str(x)[:4]) + df_hist['weekofyear'].map(lambda x: '-0' + str(x) if len(str(x))==1 else '-' + str(x) )
# df_hist.head()

### 連続week, month利用回数、期間内week, monthカバー率を特徴にする

In [13]:
df = auth0
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
train_test = pd.concat([df_train, df_test], axis=0)

for df, fname in zip([auth1, auth0], ['auth1', 'auth0']):

#     if fname.count('auth1'):
        
    df.sort_values(by=[key, 'purchase_date'], inplace=True)
    
    df_month = df[[key, 'yyyymm']].drop_duplicates()
    df_month['yyyymm_shift1'] = df_month.groupby(key)['yyyymm'].shift(1)
    df_month['diff_month'] = ((pd.to_datetime(df_month['yyyymm']) - pd.to_datetime(df_month['yyyymm_shift1'])).dt.days)//28
    # 連続してる時はフラグ1がたち、それ以外は0
    df_month['continuours_flg'] = df_month['diff_month'].where(df_month['diff_month']==1, 0)
    
    # 0になったらしきり直しなので1, 1が続く限り+1していけば、idなど関係なく、1が続いた数をカウントできる。（天才的）
    cont_list = df_month['continuours_flg'].values
    continuous_list = []
    for i in cont_list:
        if i==0:
            cnt=1
        else:
            cnt+=1
        continuous_list.append(cnt)
    print(len(cont_list))
    print(len(continuous_list))
    
    df_month['continuous'] = continuous_list
    df_month['continuous_shift1'] = df_month['continuous'].shift(1)
    df_month['diff_cont'] = df_month['continuous'] - df_month['continuous_shift1']
    # 最大連続回数を取り出す時はこれが必要
    df_month['diff_cont_shift-1'] = df_month['diff_cont'].shift(-1)
    
    # 1月以外の連続した回数を残す
    df_month.fillna(1.0, inplace=True)
    df_month_cont = df_month[df_month['diff_cont_shift-1']<0][[key, 'yyyymm', 'continuous']]
    df_month_cont.rename(columns={'continuous':'continuous_month'}, inplace=True)
    df_month_cont.head()
    
    
    cont_max = df_month_cont.groupby(key)['continuous_month'].max().reset_index()
    cont_min = df_month_cont.groupby(key)['continuous_month'].min().reset_index().rename(columns={'continuous_month':'continuous_month_min'})
    cont_mean = df_month_cont.groupby(key)['continuous_month'].mean().reset_index().rename(columns={'continuous_month':'continuous_month_mean'})
    cont_std = df_month_cont.groupby(key)['continuous_month'].std().reset_index().rename(columns={'continuous_month':'continuous_month_std'})
    
    tmp_max = df_month_cont.merge(cont_max, how='inner', on=[key, 'continuous_month'])
    cont_max_month = tmp_max.groupby(key)['yyyymm'].max()
    
    cont_max.rename(columns={'continuous_month':'continuous_month_max'}, inplace=True)
    
    cont_max_month = cont_max_month.to_frame().reset_index().rename(columns={'index':key})
    
    cont_max_month = cont_max_month.merge(train_test[[key, 'first_active_month']], how='left', on=key)
    cont_max_month = cont_max_month.merge(cont_max, how='left', on=key)
    
    cont_max_month['diff_cont_max_month_from_today'] = ((datetime.datetime.today() - pd.to_datetime(cont_max_month['yyyymm'])).dt.days)//27
    cont_max_month['diff_cont_max_last_month_from_first'] = ((pd.to_datetime(cont_max_month['yyyymm']) - pd.to_datetime(cont_max_month['first_active_month']) ).dt.days)//30
    cont_max_month['diff_cont_max_from_first'] = cont_max_month['diff_cont_max_last_month_from_first'] - cont_max_month['continuous_month_max']
    cont_max_month['diff_cont_max_from_first'] += 1
    
    cont_cnt = df_month_cont.groupby(key).size()
    cont_cnt.name = 'cont_month_cut_cnt'
    cont_cnt = cont_cnt.to_frame().reset_index().rename(columns={'index':key})
    
    
    # contがついてるfeatureのみ保存すればOK
    for feat in [cont_min, cont_mean, cont_std, cont_max_month, cont_cnt]:
        feat.columns = [col if col==key else f"{fname}_{col}" for col in feat.columns]
        df_train = df_train.merge(feat, how='left', on=key)
        df_test = df_test.merge(feat, how='left', on=key)

100%|██████████| 3/3 [00:00<00:00, 167.90it/s]


970897
970897


### 連続月達成時期を1月ごとに横持ちする（01）で
2ヵ月連続を達成したのは何か月前か？という形で  

In [ ]:
# tmp_list = []
# for i in range(2, 25, 1):
#     tmp = df_cont[df_cont['continuous_month']>=i].groupby(key)['yyyymm'].max().reset_index()
#     tmp[f'diff_month_cont_month'] = (datetime.datetime.today() - pd.to_datetime(tmp['yyyymm'])).dt.days//30
#     tmp[f'cont_month_timing_from_today'] = f'cont_{i}month_timing_from_today'
#     tmp_list.append(tmp[[key, f'cont_month_timing_from_today', f'diff_month_cont_month']].copy())
# cont_from = pd.concat(tmp_list, axis=0)
# display(cont_from.head(10))
# display(cont_from.tail(10))

In [15]:
# cont_from_table = cont_from.pivot_table(index=key, columns='cont_month_timing_from_today', values='diff_month_cont_month')
# cont_from_table.head()
# df_train = df_train.merge(cont_from_table, how='left', on=key)
# df_test = df_test.merge(cont_from_table, how='left', on=key)
# print(df_train.shape)
# df_train.head()

In [16]:
df_train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,auth1_continuous_month_min,auth1_continuous_month_mean,auth1_continuous_month_std,auth1_yyyymm,auth1_first_active_month,auth1_continuous_month_max,auth1_diff_cont_max_month_from_today,auth1_diff_cont_max_last_month_from_first,auth1_diff_cont_max_from_first,auth1_cont_month_cut_cnt,auth0_continuous_month_min,auth0_continuous_month_mean,auth0_continuous_month_std,auth0_yyyymm,auth0_first_active_month,auth0_continuous_month_max,auth0_diff_cont_max_month_from_today,auth0_diff_cont_max_last_month_from_first,auth0_diff_cont_max_from_first,auth0_cont_month_cut_cnt
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,9.0,9.0,NaN,2018-02,2017-06,9.0,13.0,8.0,0.0,1.0,6.0,6.0,NaN,2017-12,2017-06,6.0,15.0,6.0,1.0,1.0
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,13.0,13.0,NaN,2018-01,2017-01,13.0,14.0,12.0,0.0,1.0,2.0,2.0,NaN,2017-10,2017-01,2.0,17.0,9.0,8.0,1.0
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,3.0,4.0,1.0,2017-05,2016-08,5.0,23.0,9.0,5.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,6.0,6.0,NaN,2018-02,2017-09,6.0,13.0,5.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,4.0,4.0,NaN,2018-02,2017-11,4.0,13.0,3.0,0.0,1.0,2.0,2.0,NaN,2018-02,2017-11,2.0,13.0,3.0,2.0,1.0


In [18]:
fname = '134_com'
ignore_features = ['first_active_month', 'card_id']

df_feat = pd.concat([df_train, df_test], axis=0)
df_feat.fillna(-1, inplace=True)

for col in df_feat.columns:
#     if col in ignore_features:
#         continue
#     if not(col.count('feature_')):continue
    if not(col.count('cont')):continue

    feature = df_feat[col].values.astype('float32')
    if feature.shape[0]!=325540:
        print(feature.shape)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_all_{col}@', obj=feature)

(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)
(325540,)


### 135 week

In [19]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
train_test = pd.concat([df_train, df_test], axis=0)

for df, fname in zip([auth1, auth0], ['auth1', 'auth0']):

    df.sort_values(by=[key, 'purchase_date'], inplace=True)
    df_week = df[[key, 'yyyy_week']].drop_duplicates()
    
    week_val = df_week['yyyy_week'].drop_duplicates().sort_values().to_frame()
    week_val['index'] = np.arange(len(week_val)) + 1
    df_week = df_week.merge(week_val, how='inner', on='yyyy_week')
    
    df_week.sort_values(by=[key, 'yyyy_week'], inplace=True)
    df_week['index_shift1'] = df_week.groupby(key)['index'].shift(1)
    df_week['diff_week'] = df_week['index'] - df_week['index_shift1']
    # 連続してる時はフラグ1がたち、それ以外は0
    df_week['continuours_flg'] = df_week['diff_week'].where(df_week['diff_week']==1, 0)
    
    # 0になったらしきり直しなので1, 1が続く限り+1していけば、idなど関係なく、1が続いた数をカウントできる。（天才的）
    cont_list = df_week['continuours_flg'].values
    continuous_list = []
    for i in cont_list:
        if i==0:
            cnt=1
        else:
            cnt+=1
        continuous_list.append(cnt)
    print(len(cont_list))
    print(len(continuous_list))
    
    df_week['continuous'] = continuous_list
    df_week['continuous_shift1'] = df_week['continuous'].shift(1)
    df_week['diff_cont'] = df_week['continuous'] - df_week['continuous_shift1']
    # 最大連続回数を取り出す時はこれが必要
    df_week['diff_cont_shift-1'] = df_week['diff_cont'].shift(-1)
    
    # 1月以外の連続した回数を残す
    df_week.fillna(1.0, inplace=True)
    df_cont = df_week[df_week['diff_cont_shift-1']<0][[key, 'yyyy_week', 'continuous']]
    df_cont.rename(columns={'continuous':'continuous_week'}, inplace=True)
    
    
    cont_max = df_cont.groupby(key)['continuous_week'].max().reset_index().rename(columns={'continuous_week':'continuous_week_max'})
    cont_min = df_cont.groupby(key)['continuous_week'].min().reset_index().rename(columns={'continuous_week':'continuous_week_min'})
    cont_mean = df_cont.groupby(key)['continuous_week'].mean().reset_index().rename(columns={'continuous_week':'continuous_week_mean'})
    cont_std = df_cont.groupby(key)['continuous_week'].std().reset_index().rename(columns={'continuous_week':'continuous_week_std'})
    
    cont_cnt = df_cont.groupby(key).size()
    cont_cnt.name = 'cont_week_cut_cnt'
    cont_cnt = cont_cnt.to_frame().reset_index().rename(columns={'index':key})
    
    # contがついてるfeatureのみ保存すればOK
    for feat in [cont_max, cont_min, cont_mean, cont_std, cont_cnt]:
        feat.columns = [col if col==key else f"{fname}_{col}" for col in feat.columns]
        df_train = df_train.merge(feat, how='left', on=key)
        df_test = df_test.merge(feat, how='left', on=key)
        
df_train.head()

100%|██████████| 3/3 [00:00<00:00, 164.57it/s]


6712457
6712457
1343579
1343579


,first_active_month,card_id,feature_1,feature_2,feature_3,target,auth1_continuous_week_max,auth1_continuous_week_min,auth1_continuous_week_mean,auth1_continuous_week_std,auth1_cont_week_cut_cnt,auth0_continuous_week_max,auth0_continuous_week_min,auth0_continuous_week_mean,auth0_continuous_week_std,auth0_cont_week_cut_cnt
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,35.0,35.0,35.000000,NaN,1.0,3.0,2.0,2.333333,0.57735,3.0
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,24.0,8.0,13.500000,7.325754,4.0,4.0,4.0,4.000000,NaN,1.0
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,7.0,2.0,3.750000,2.362908,4.0,NaN,NaN,NaN,NaN,NaN
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,11.0,3.0,6.333333,4.163332,3.0,NaN,NaN,NaN,NaN,NaN
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,17.0,17.0,17.000000,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [20]:
fname = '135_cow'
ignore_features = ['first_active_month', 'card_id']

df_feat = pd.concat([df_train, df_test], axis=0)
df_feat.fillna(-1, inplace=True)

for col in df_feat.columns:
#     if col in ignore_features:
#         continue
#     if not(col.count('feature_')):continue
    if not(col.count('cont')):continue

    feature = df_feat[col].values.astype('float32')
    if feature.shape[0]!=325540:
        print(feature.shape)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_all_{col}@', obj=feature)